Segunda prueva realizada.
Se realizan una regresión logistica, un random forest y un gradient boosting.
Los datos usados son los del dataset KOI tomando:
- Como output se toma la columns "koi_disposition" con "Confirmed" como 1 y "FALSE POSITIVE" como 0.
- Como input se toman todas las columnas excepto "koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec","koi_tce_delivname", que no son datos numéricos y exepto "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_plnt_num" que no son datos relevantes.

La diferencia con la primera prueba es que en este caso si que usamos los errores considerados por el dataset.

In [7]:
import pandas as pd
import numpy as np
dataPath = "cumulative_2025.09.26_02.47.52.csv"

In [8]:
df_completo = pd.read_csv(dataPath)
df_out_filtrado = df_completo[df_completo["koi_disposition"].isin(["CONFIRMED", "FALSE POSITIVE"])]

In [9]:
#Eliminación de columnas datos no numéricos
df_out_filtrado = df_out_filtrado.drop(columns=["koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec","koi_tce_delivname"])

#Eliminación de columnas irrelevantes
df_out_filtrado = df_out_filtrado.drop(columns=["kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_plnt_num"])
#koi_tce_plnt_num es cuantos planetas ha detectado con una señal

#Eliminación de columnas vacias
df_out_filtrado = df_out_filtrado.drop(columns=["koi_teq_err1", "koi_teq_err2"])


# Eliminacions de NaNs
df_out_filtrado = df_out_filtrado.dropna()
#df_out_filtrado = df_out_filtrado.fillna(df_out_filtrado.mean(numeric_only=True)) #Por si se quiere reemplazar con la madia en vez de eliminar
#La reduccion es de solo 250 datos, creo que renta eliminarlos para no crear datos falsos

In [10]:
df_out_filtrado

,koi_disposition,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
3,FALSE POSITIVE,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
5,CONFIRMED,11.094321,2.036000e-05,-2.036000e-05,171.201160,0.001410,-0.001410,0.538,0.030,-0.428,...,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9557,FALSE POSITIVE,373.893980,1.661000e-02,-1.661000e-02,261.496800,0.034900,-0.034900,0.963,0.067,-0.658,...,-141.0,4.574,0.078,-0.052,0.699,0.074,-0.066,290.93597,47.290730,14.911
9558,FALSE POSITIVE,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,0.765,0.023,-0.541,...,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9559,FALSE POSITIVE,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,1.252,0.051,-0.049,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,FALSE POSITIVE,0.681402,2.434000e-06,-2.434000e-06,132.181750,0.002850,-0.002850,0.147,0.309,-0.147,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [11]:
y = df_out_filtrado["koi_disposition"].values
y = (y == "CONFIRMED")
X = df_out_filtrado.drop(columns=["koi_disposition"]).values

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score

### Regresión logistica

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_n = sc.fit_transform(X_train)
lr = LogisticRegression()
lr.fit(X_train_n, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [15]:
X_test_n = sc.transform(X_test)
print(accuracy_score(y_test,lr.predict(X_test_n)))
print(precision_score(y_test,lr.predict(X_test_n)))
print(f1_score(y_test,lr.predict(X_test_n)))

0.9167728237791932
0.8846560846560847
0.8950749464668094


In [16]:
lr.coef_

array([[-0.76790669, -0.02421467,  0.02421467, -0.03546603,  0.05481027,
        -0.05481027, -5.4507805 , -1.10945743, -1.84172668, -0.26608122,
        -1.98301337,  1.98301337, -6.76715967, -0.32694606,  0.32694606,
        -0.32316341, -0.71318894,  0.13883969, -2.84311652,  0.16079069,
         0.32357826, -0.29291904, -0.35776188,  1.10234375, -0.64376808,
         0.64623731, -0.90181769,  0.41224921,  0.09251185, -0.61680844,
        -2.06478757,  1.20459203, -0.2342678 ,  0.29028566,  0.40188532]])

### Random forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_n, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [18]:
X_test_n = sc.transform(X_test)
print(accuracy_score(y_test,rf.predict(X_test_n)))
print(precision_score(y_test,rf.predict(X_test_n)))
print(f1_score(y_test,rf.predict(X_test_n)))

0.9320594479830149
0.9340159271899886
0.9112097669256382


### Gradient boosting

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
gbr = GradientBoostingClassifier(
    n_estimators=100,    # número de árboles
    learning_rate=0.1,   # factor de aprendizaje (shrinkage)
    max_depth=8,         # profundidad de cada árbol
    random_state=42
)
gbr.fit(X_train_n, y_train)

,loss,'log_loss'
,learning_rate,0.1
,n_estimators,1000
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,8
,min_impurity_decrease,0.0
,init,None


In [26]:
X_test_n = sc.transform(X_test)

In [27]:
print(accuracy_score(y_test,gbr.predict(X_test_n)))
print(precision_score(y_test,gbr.predict(X_test_n)))
print(f1_score(y_test,gbr.predict(X_test_n)))

0.9414012738853503
0.9308452250274424
0.9247546346782988
